# Graphen und Netzwerke.

Algorithmen zu Spannende Bäume gehören überarbeitet!!!!


In [ ]:
import sys
import os
import string
import time
import re
import datetime
import codecs
import itertools
import numpy as np
import sympy as sp
import pycurl
import matplotlib.pyplot as plt

from copy import deepcopy
from scipy.special import binom
from matplotlib import cm
from matplotlib.ticker import LinearLocator

# Ausgabe von Rechnername und Datum:
hostname = os.uname().nodename.split('.')[0].capitalize()
last_started = datetime.date.today()
print(f"Notebook wurde zuletzt gestartet {last_started} am Rechner {hostname}.")

In [ ]:
# OMIT Moodle-Output ausschalten

In [ ]:
# Selbstgebastelte Module:
from mf.lecture import write_code_snippet, write_function_snippet
from mf.texutils import find_occurrences

# Root folder:
ROOTPATH = '/Users/mfulmek/ucloud/books/dmti/'

# Name und Nummer des Files für das aktuelle Kapitel:
NUMBER = 4
SECTION = 'graphs'
SOURCE = os.path.join(ROOTPATH, "snippets", f'{SECTION}_snippets.ipynb')

# Keyword-Arguments für die Funktionsaufrufe:
COMMON = {
    'rootpath' : ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : False
}

print(f"Snippets für Kapitel 4: Graphen und Netzwerke.")

In [ ]:
# MOODLE Moodle-Output einschalten

## Darstellung eines Graphen in Python

Eine einfache Möglichkeit für einfache (ungerichtete) Graphen ist die _Adjazenzmatrix_:

### Hilfsfunktion: Kante in die Adjazenzmatrix eines ungerichteten Graphen eintragen.

In [ ]:
def modify_edge(am, i, j, entry=1):
    """Trage eine Kante in einer Adjazenzmatrix ein: Die Matrix
    soll _symmetrisch_ sein."""
    try:
        am[i][j] = am[j][i] = entry
    except IndexError:
        print(f'Indices ({i},{j}) nicht in {am.shape}!')

In [ ]:
write_function_snippet(modify_edge,
    **COMMON,
    the_caption=r'Modifiziere Kante in der Adjazenzmatrix eines ungerichteten Graphen.',
    preamble = ''
)

### Erzeuge die Adjazenzmatrix eines ungerichteten Graphen aus einer Liste von Kanten.

In [ ]:
def adjacency_matrix_from_edge_set(n, edges):
    """Erzeuge die Adjazenzmatrix eines Graphen mit n Knoten
    und vorgegebener Kantenmenge"""
    am = np.zeros(n**2, dtype=int).reshape((n,n))
    for i,j in edges:
        modify_edge(am, i, j)
    return am

In [ ]:
write_function_snippet(adjacency_matrix_from_edge_set,
    **COMMON,
    the_caption=r'Erzeuge Adjazenzmatrix eines ungerichteten Graphen aus Liste von Kanten.',
    preamble = ''
)

### Rekonstruiere die Kantenmenge eines Graphen aus seiner Adjazenzmatrix.


In [ ]:
def get_edges(am):
    """Rekonstruiere die Kantenmenge eines Graphen aus seiner
    (quadratischen und symmetrischen!) Adjazenzmatrix;
    retourniere diese in einer Liste."""
    # Anzahl der Knoten:
    n = am.shape[0]
    # Liste der bereits identifizierten Kanten:
    edges = []
    for i in range(n-1):
        for j in range(i+1,n):
            if am[i][j]:
                edges+= [(i,j)]
    return edges

In [ ]:
write_function_snippet(get_edges,
    **COMMON,
    the_caption=r'Rekonstruiere die Kantenmenge eines Graphen aus seiner Adjazenzmatrix.',
    preamble = ''
)

## Finde Zusammenhangskomponenten in einem (einfachen) Graphen.
 Für einen (einfachen) Graphen, der durch seine
 Adjazenzmatrix gegeben ist, finden wir durch
 _direktes Suchen_ seine Komponenten und retournieren
 diese als Liste von (Knoten-)Mengen.

In [ ]:
def get_components(am):
    """Finde die Zusammenhangskomponenten des Graphen mit
    (quadratischer und symmetrischen!) Adjazenzmatrix am;
    retourniere diese in einer Liste."""
    #@ Für einen (einfachen) Graphen, der durch seine
    #@ Adjazenzmatrix gegeben ist, finden wir durch
    #@ ``direktes Suchen'' seine Komponenten und retournieren
    #@ diese als Liste von (Knoten-)Mengen.
    # Anzahl der Knoten:
    n = am.shape[0]
    # Liste der bereits gefundenen Zusammenhangskomponenten
    components = []
    # Menge der (Indizes von) Knoten, die noch nicht auf
    # Zusammenhangskomponenten verteilt wurden:
    vertices = set(range(n))
    # Solange vertices nicht leer ist:
    while vertices:
        # Wir suchen eine neue Komponente:
        new_component = set()
        # Wir geben "schichtenweise Knoten" zur neuen Komponente,
        # die "innerste Schicht" ist einpunktig:
        current_layer = set([min(vertices)])
        # Solange current_layer nicht leer ist:
        while current_layer:
            # Die "neue Schicht" kommt zur neuen Komponente dazu ...
            new_component.update(current_layer)
            # ... und wird von der Knotenmenge weggenommen:
            vertices.difference_update(current_layer)
            # Jetzt suchen wir die "nächste Schicht":
            # print(components, new_component, vertices)
            new_layer = set()
            # Mit der itertools-Funktion "product" durchlaufen wir alle
            # möglichen Kanten und schauen, ob die im Graphen vorhanden
            # sind:
            for i,j in itertools.product(current_layer, vertices):
                if am[i][j]:
                    new_layer.add(j)
            current_layer = new_layer
        # Nach der inneren while-Schleife speichern wir new_component:
        components+= [new_component]
    # Nach der äußeren while-Schleife retournieren wir die Komponenten:
    return components

In [ ]:
write_function_snippet(get_components,
    **COMMON,
    the_caption=r'Finde Zusammenhangskomponenten in einem Graphen.',
    preamble = ''
)

In [ ]:
# OMIT Ausarbeitung eines Übungsbeispiels

In [ ]:
def is_connected(amat):
    """Test: Ist der durch die Adjazenzmatrix amat gegebene Graph zusammenhängend?"""
    n = amat.shape[0]
    if n <= 1:
        # Der "leere Graph" und der einpunktige Graph sind zusammenhängend
        return True
    # Implicit else:
    return len(get_component(amat, 0)) == n

In [ ]:
def is_eulerian(amat):
    """Teste, ob die (symmetrische) Adjazenmatrix amat einen Eulerschen
    Graphen bestimmt."""
    if not is_connected(amat):
        return False
    # Implicit else:
    return (np.sum(amat, axis=0, dtype=int)%2 == 0).all()

In [ ]:
def get_component(amat, v):
    """Bestimme die Liste der Knoten, die zur selben Zusammenhangskomponente
    gehören wie der Knoten v. (Der Graph ist - wie immer - durch seine Adjazenzmatrix
    amat gegeben.)"""
    # v gehört natürlich zu "seiner eigenen" Komponente:
    component = [v]
    # Am Anfang ist v außerdem der einzige Knoten, von dem
    # aus die Komponente "potentiell erweitert" werden kann:
    pot_ext = [v]
    # Im Verlauf des einfachen Algorithmus werden "Kanten gestrichen",
    # also die Adjazenzmatrix modifiziert: Dafür machen wir eine _Kopie_
    # der "originalen" Adjazenzmatrix (die ja vielleicht anderweitig
    # noch gebraucht wird.)
    graph = np.copy(amat)
    # Solange es Knoten gibt, von denen aus die Komponente
    # "potentiell erweitert" werden kann, probieren wir das:
    while pot_ext:
        # Der Befehl pop entfernt das letzte Element der Liste
        # und liefert es als Wert zurück:
        u = pot_ext.pop()
        # Suche die Nachbarn von u (als Liste):
        neighbours = list(nachbarn(graph, u))
        # Entferne alle Kanten, die _irgendwelche_ Knoten aus der Liste
        # component+neighbours verbinden: Damit ist sichergestellt, daß im
        # nächsten Schritt keine Nachbarn gefunden werden, die bereits
        # zur Komponente dazugefügt wurden!
        for i,j in itertools.combinations(component+neighbours,2):
            modify_edge(graph, i, j, 0)
        # Diese ("außerhalb der Komponente liegenden" Nachbarn!!!)
        # erweitern die Komponente und die "potentiell erweiternden"
        # Knoten:
        component+= neighbours
        pot_ext+= neighbours
        
    return component

In [ ]:
def eulerian_trail(amat):
    """Finde eine Eulersche Wanderung."""
    if not is_eulerian(amat):
        return None
    # Implicit else:
    # Wir machen wieder eine Kopie von amat, in der wir Kanten
    # löschen werden:
    graph = np.copy(amat)
    # Wir merken uns die Anzahl der Knoten:
    n = amat.shape[0]
    # Initialisierung: Wir beginnen mit dem Knoten 0
    v_a = 0
    trail = [v_a]
    # list(nachbarn) liefert eine Liste; deren Wahrheitswert
    # ist False, wenn sie leer ist - und das ist genau das, was
    # wir hier brauchen:
    neighbours = list(nachbarn(graph, v_a))
    while neighbours:
        if len(neighbours) == 1:
            v_b = neighbours[0]
            # Das ist die Situation, wo die Wegnahme der Kante
            # (v_a, v_b) _zwei_ Zusammenhangskomponenten erzeugt,
            # von denen eine nur aus v_a besteht: Wir löschen die
            # Kante (später) trotzdem und merken uns, daß wir ab nun
            # "einen Knoten weniger" in dem Graphen haben
            n-= 1
        else:
            # Wir prüfen für alle benachbarten Knoten v_b, ob die Wegnahme
            # der Kante den Graphen (auf den noch vorhandenen Knoten!!!)
            # einen zusammenhängenden Graphen ergibt:
            success = False
            for v_b in neighbours:
                # Entferne (versuchsweise) die Kante:
                modify_edge(graph, v_a, v_b, 0)
                # Der Graph G insgesamt ist vielleicht _nicht mehr_ zusammenhängend
                # (er kann einpunktige Komponenten haben, siehe if-clause!) -
                # aber die Komponente von v_b sollte alle n  "noch relevanten" Knoten
                # enthalten!!!
                success = len(get_component(amat, v_b)) == n
                if success:
                    break
                # Implicit else:
                # Ok, nächster Versuch - gib Kante wieder dazu:
                modify_edge(graph, v_a, v_b, 1)
            if not success:
                print(f'Oops!')
                break
        # Ok: Verlängere die Wanderung ...
        trail+=[v_b]
        # ... und entferne die Kante (v_a, v_b)
        modify_edge(graph, v_a, v_b, 0)
        # Bereite den nächsten Durchlauf der Schleife vor:
        v_a = v_b
        neighbours = list(nachbarn(graph, v_a))
        
    return trail

In [ ]:
# MOODLE wieder einschalten

## Spannende Bäume in einem Graphen

### Hilfsfunktion: Finde verbindende Kanten zwischen Zusammenhangskomponenten.

 Für eine Liste von Zusammenhangskomponenten suchen wir
 alle Kanten aus einer Liste, die zwei Komponenten verbinden,
 durch "direktes Suchen" und retournieren das Ergebnis
 in Form einer Liste, die für jede Kante die verbundenen
 Komponenten zeigt.

In [ ]:
def get_connecting_edges(list_of_components,list_of_edges):
    """Finde alle Kanten in der Menge list_of_edges, die
    zwei verschiedene Komponenten aus der Menge list_of_components
    verbinden:"""
    #@ Für eine Liste von Zusammenhangskomponenten suchen wir
    #@ alle Kanten aus einer Liste, die zwei Komponenten verbinden,
    #@ durch ``direktes Suchen'' und retournieren das Ergebnis
    #@ in Form einer Liste, die für jede Kante die verbundenen
    #@ Komponenten zeigt.
    result = []
    for na, a in enumerate(list_of_components[:-1]):
        for nb, b in enumerate(list_of_components[na+1:], start=na+1):
            for ne,(i,j) in enumerate(list_of_edges):
                if ((i in a) and (j in b) or (j in a) and (i in b)):
                    result+=[(ne,(na,nb))]
    return result

In [ ]:
write_function_snippet(get_connecting_edges,
    **COMMON,
    the_caption=r'Finde verbindende Kanten zwischen Zusammenhangskomponenten.',
    preamble = ''
)

### Finde spannenden Baum.

In [ ]:
def find_spanning_tree(am):
    """Finde einen spannenden Baum: Umsetzung des Algorithmus."""
    n = am.shape[0]
    edges = get_edges(am)
    
    S = []
    T = adjacency_matrix_from_edge_set(n, S)
    
    loc = get_components(T)
    while len(loc) > 1:
        connectors = get_connecting_edges(loc, edges)
        if connectors:
            i,j = edges[connectors[0][0]]
            modify_edge(T,i,j)
            S+= [(i,j)]
        else:
            print('Graph ist nicht zusammenhängend!')
            return
        loc = get_components(T)
    return S,T

In [ ]:
write_function_snippet(find_spanning_tree,
    **COMMON,
    the_caption=r'Finde spannenden Baum.',
    preamble = ''
)

### Implementierung des spanning--tree--algorithmus mit _functions of restricted growths_.

Das ist eine viel elegantere Implementierung!?

Die Zusammenhangskomponenten eines Graphen $G$ bilden natürlich eine _(Mengen-)Partition_ seiner Knotenmenge $V(G)$, die wir mit einer _function of restricted growth_ beschreiben können:

In [ ]:
def find_spanning_tree2(adj_matrix):
    """Algorithmus zur Konstruktion eines spannenden Baumes"""
    # adj_matrix sollte die Adjazenzmatrix des (einfachen) Graphen
    # sein: Eintrag 0 bedeutet, daß die entsprechende
    # Kante nicht existiert.
    dim = adj_matrix.shape[0]
    # Zu Beginn gibt es keine Kante im spannenden Wald ...
    list_of_used_edges = []
    # ... und jeder Knoten ist ein Singleton-Block; die Blöcke (Komponenten)
    # werden mit einer _Funktion von beschränktem Wachstum_ codiert; zu
    # Beginn also durch (0,1,...,dim-1):
    fn_restricted_growth = np.arange(dim, dtype=int)
    # Die Kanten des Graphen entsprechen den nicht-Null-Einträgen _strikt
    # oberhalb_ der Hauptdiagonale; wir extrahieren sie aus der
    # Adjazenzmatrix.
    # Zuerst bestimmen wir einmal die _Anzahl_ der Kanten; mithilfe von
    # numpy-Funktionen: triu(m,j) liefert die Dreiecksmatrix "j Diagonalen
    # oberhalb der Hauptdiagonalen von m", und count_nonzero(matrix) zählt
    # die Einträge in matrix, die nicht Null sind:
    nof_edges = np.count_nonzero(np.triu(adj_matrix,1))
    
    # Vorbereitung von arrays, die die Kanten-Informationen aufnehmen
    # sollen:
    list_of_edges = np.zeros(2*nof_edges, dtype=int).reshape(nof_edges, 2)
    # "Auslesen" der Kanten aus der Adjazenzmatrix:
    k = 0
    for i in range(dim):
        for j in range(i+1,dim):
            entry = adj_matrix[i][j]
            if entry:
                list_of_edges[k][0] = i
                list_of_edges[k][1] = j
                k+= 1
    
    # Jetzt der eigentliche Algorithmus:
    for (a,b) in list_of_edges:
        # Die Funktion von beschränktem Gewicht codiert, zu welchen
        # Blöcken (Komponenten) die Endpunkte a, b gehören:
        block_no_1 = fn_restricted_growth[a]
        block_no_2 = fn_restricted_growth[b]
        # Verbindet Kante (a,b) zwei Knoten in derselben Komponente?
        if block_no_1 == block_no_2:
            # Ja: Überspringe den Rest der Schleife!
            continue
        # Implicit else:
        # Nein, diese Kante verbindet zwei _verschiedene_ Komponenten!
        # Wir verwenden sie also für den spannenden Wald:
        list_of_used_edges+= [[a,b]]
        # Die neu verbundenen Komponenten werden nun zu einer neuen
        # Komponenten "verschmolzen", das müssen wir in
        # fn_restricted_growth abbilden:
        if block_no_2 < block_no_1:
            dummy = block_no_2
            block_no_2 = block_no_1
            block_no_1 = dummy
        # Nun gilt jedenfalls block_no_1 < block_no_2
        # Wieder numpy-Funktionalität: "fn_restricted_growth==block_no_2"
        # liefert jenes Array von _indices_ i, für die 
        # fn_restricted_growth[i]==block_no_2 gilt; die werden dann mit
        # block_no_1 überschrieben:
        fn_restricted_growth[
            np.nonzero(fn_restricted_growth==block_no_2)
        ] = block_no_1
        # Mit derselben numpy-Technik: Alle Funktionswerte von
        # fn_restricted_growth, die größer als block_no_2 sind, werden
        # um 1 vermindert:
        for k in range(block_no_2+1,np.max(fn_restricted_growth)+1):
            fn_restricted_growth[np.nonzero(fn_restricted_growth==k)]-=1

    # Rückgabe: Kanten des minimalen spannenden Waldes und
    # Zusammenhangskomponenten (als Partition der Knotenmenge;
    # codiert als Funktion von beschränktem Wachstum):
    return list_of_used_edges, fn_restricted_growth

In [ ]:
write_function_snippet(find_spanning_tree2,
    **COMMON,
    the_caption=r'Implementierung: Spanning--Tree--Algorithmus.',
    preamble = ''
)

## Zusammenhangskomponenten mit Modul ```scipy```

Im Modl ```scipy``` stehen (unter anderem) Funktionen zur Erzeugung von Graphen und zur Auffindung von Zusammenhangskomponenten zur Verfügung.

In [ ]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
graph = [
    [ 0, 1 , 1, 0 , 0 ],
    [ 0, 0 , 1 , 0 ,0 ],
    [ 0, 0, 0, 0, 0],
    [0, 0 , 0, 0, 1],
    [0, 0, 0, 0, 0]
]
graph = csr_matrix(graph)
n_components, labels = connected_components(
    csgraph=graph,
    directed=False,
    return_labels=True
)
print(f'Der Graph hat {n_components} Zusammenhangskomponenten.')

In [ ]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'scipy-graph-components',
    **COMMON,
    the_caption = r'Zusammenhangskomponenten mit \pythoncode{scipy} (nur zur Illustration).'
    # preamble = """from scipy.sparse import csr_matrix
# from scipy.sparse.csgraph import connected_components"""
)